In [ ]:
from ddpm import DDPMSampler

from PIL import Image
import torch
import numpy as np
import math

In [ ]:
generator = torch.Generator()
generator.manual_seed(0)

In [ ]:
ddpm_sampler = DDPMSampler(generator)

In [ ]:
# How many noise levels to generate
noise_levels = [0, 10, 50, 75, 100, 250, 500, 750]

In [ ]:
img = Image.open("../images/dog.jpg")

In [ ]:
img_tensor = torch.tensor(np.array(img))
img_tensor = ((img_tensor / 255.0) * 2.0) - 1.0
img_tensor.shape

In [ ]:
from torchvision import transforms
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor (scales to [0, 1])
    # This is basically 2x - 1 which is what manual approach does
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

image_tensor = transform(img)
image_tensor.shape
image_tensor = image_tensor.permute(1, 2, 0)
image_tensor.shape

In [ ]:
torch.equal(image_tensor, img_tensor)

In [ ]:
img_batch = image_tensor.repeat(len(noise_levels), 1, 1, 1)
img_batch.shape

In [ ]:
ts = torch.tensor(noise_levels)
noise_imgs = []
epsilons = torch.randn(img_batch.shape)

In [ ]:
# Add varying degrees of noise to the image
for i in range(len(ts)):
    a_hat = ddpm_sampler.alphas_cumprod[ts[i]]
    noise_img = (math.sqrt(a_hat) * img_batch[i]) + (math.sqrt(1 - a_hat) * epsilons[i])
    noise_imgs.append(noise_img)

In [ ]:
noise_imgs[0].shape

In [ ]:
noise_imgs = torch.stack(noise_imgs, dim=0)
noise_imgs.shape

In [ ]:
noise_imgs = (noise_imgs.clamp(-1, 1) + 1) / 2 # normalize to [0, 1]
noise_imgs = (noise_imgs * 255).type(torch.uint8) # [0, 255] which is the normal range

In [ ]:
noise_imgs.dtype

In [ ]:
# Now display images
display_img = Image.fromarray(noise_imgs[1].numpy(), 'RGB')
display_img